OpenVINO 준비

In [1]:
%pip install -q "openvino>=2023.1.0"

In [2]:
!pip install openvino-dev

keras .tf 모델을 Intel OpenVINO IR 파일(xml, bin)로 한다.

In [4]:
import openvino
import openvino as ov
from openvino.tools.mo import convert_model

BASE_PATH = './drive/MyDrive/Colab Notebooks/data/'
ov_model = convert_model(BASE_PATH+'bin/mobilenetv2_class4.tf')
ov.save_model(ov_model, BASE_PATH+'bin/mobilenetv2_class4.xml')

### 테스트 파일 목록으로 읽어와서 답안 출력하기.



In [5]:
# --------------------------- Step 1. Initialize OpenVINO Runtime Core ------------------------------------------------
core = ov.Core()

In [6]:
# --------------------------- Step 2. Read a model --------------------------------------------------------------------
# (.xml and .bin files) or (.onnx file)
model = core.read_model(BASE_PATH+'bin/mobilenetv2_class4.xml')

if len(model.inputs) != 1: print('Sample supports only single input topologies')
if len(model.outputs) != 1: print('Sample supports only single output topologies')

In [7]:
import os

directory_path = "./drive/MyDrive/Colab Notebooks/data/valid"
#directory_path = "./drive/MyDrive/Colab Notebooks/data/test"

# Get a list of all files in the directory
file_list = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# Print the list of files
print("List of files in the directory:")
for file in file_list:
    print(file)

List of files in the directory:
v83u6a.jpg
PQ1oYj.jpg
6fe557.jpg
TQ5Ms3.jpg
1ZJ90p.jpg
HUu_C_.jpg
2XM4f3.jpg
c2a8r9.jpg
8lkMKe.jpg
X5_262.jpg
q_2Mv_.jpg
__96D_.jpg
kH3xjZ.jpg
0B02VC.jpg
gwT5N2.jpg
_2J4_e.jpg
uPew9g.jpg
5x8x1v.jpg
42Ucd5.jpg
mX2_I_.jpg
qV_80L.jpg
S4P4U8.jpg
grfG9G.jpg
gP_M87.jpg
D4tkP9.jpg
0m_wUL.jpg
g_06mI.jpg
_l17Ag.jpg
ZhJEqW.jpg
_o_7_t.jpg
DGyK8O.jpg
5JmtGT.jpg
l_Sa9O.jpg
Lz8ebf.jpg
KKDs3N.jpg
WOAkd_.jpg
1TKm_N.jpg
4R9KsW.jpg
8ht_7M.jpg
_bT8u5.jpg
B208B1.jpg
4Xrr5_.jpg
_8T9Ce.jpg
_TgqIQ.jpg
pi358W.jpg
__XSg3.jpg
17yo_A.jpg
f78CMW.jpg
02x_DZ.jpg
HVQT9B.jpg
6_FL4m.jpg
p_E3Mv.jpg
rZD47a.jpg
EbC_bu.jpg
137492.jpg
Bo_79j.jpg
y5WgM5.jpg
4F4jPx.jpg
MDF7_5.jpg
SJNAwN.jpg
7Te_Z4.jpg
j85_lJ.jpg
7_md1M.jpg
3G3AeY.jpg
L_w5Ko.jpg
z_IhOf.jpg
wH0rVG.jpg
F_6B38.jpg
y9_c2n.jpg
H15wl0.jpg
AnYmT3.jpg
vZ9o9T.jpg
0N8A9_.jpg
V32m32.jpg
hvro2h.jpg
eIa3GJ.jpg
VMK4_e.jpg
S1g8m2.jpg
Z0N_EM.jpg
5nqLzT.jpg
mE19_d.jpg
7DL3S4.jpg
E6F9E6.jpg
_0Y_T9.jpg
DEJGmE.jpg
7732Sa.jpg
G_e_0V.jpg
G_YO6_.jpg


In [8]:
import pickle

with open(BASE_PATH+'bin/class4.pickle', 'rb') as f:
    labels = pickle.load(f)

파일 추론하기

In [12]:
import pandas as pd
import cv2
import numpy as np

# --------------------------- Step 4. Loading model to the device -----------------------------------------------------
print('Loading the model to the plugin')
compiled_model = core.compile_model(model, 'CPU')
img_height = 224

result = []

res = open(BASE_PATH+"result.txt", "w")

print('Starting inference in synchronous mode')
cnt = 0;

for file in file_list:
    # --------------------------- Step 5. Set up input --------------------------------------------------------------------
    # Read input image
    image_path = directory_path + "/" + file
    org_image = cv2.imread(image_path)
    image = cv2.resize(org_image, (img_height,img_height), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
    #img = cv2.resize(img, (224,224), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.

    # Add N dimension
    nchw_tensor = np.expand_dims(image, 0)

    # NHWC -> NCHW
    input_tensor = np.transpose(nchw_tensor, (0,1,2,3))#(0,3,1,2))

    # --------------------------- Step 6. Create infer request and do inference synchronously -----------------------------
    results = compiled_model.infer_new_request({0:input_tensor})

    # --------------------------- Step 7. Process output ------------------------------------------------------------------
    predictions = next(iter(results.values()))
    output_node_name = next(iter(results.keys())) #
    #for (k,v) in results.items():
    #  print(k,'=',v)

    #print("values: ",end=' ')
    #print(results.values())

    # Change a shape of a numpy.ndarray with results to get another one with one dimension
    probs = predictions.reshape(-1)
    #print(outs)
    #print("probs: ",end=' ')
    #print(probs)

    # Get an array of 4 class IDs in descending order of probability
    top_4 = np.argsort(probs)[-4:][::-1]
    print(top_4)

    #res = res[output_node_name]
    #idx = np.argsort(res[0])[-1]
    #prob = res[0][idx]*100

    header = 'class_id            probability'
    print(f'Image path: {image_path}')
    print('Top 4 results: ')
    print(header)
    print('-' * len(header))

    for class_id in top_4:
        probability_indent = ' ' * (len('class_id           ') - len(labels[class_id]) + 1)
        print(f'{labels[class_id]}{probability_indent}{probs[class_id]*100:.7f}')

    #print(labels[top_4[0]], probs[top_4[0]]*100)
    #print(labels[top_4[1]], probs[top_4[1]]*100)
    #result.append(top_4[0])
    #res.write(file+':'+str(labels[top_4[0]])+'\n')
    res.write(file+':top1 = ('+labels[top_4[0]]+')'+str(probs[top_4[0]]*100)+', top2 = ('+labels[top_4[1]]+')'+str(probs[top_4[1]]*100)+'\n')

#res.write(str(result)+'\n')
res.close()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
-------------------------------
Pneumonia           99.2567182
NoPneumonia         0.7396145
NoOsteoarthritis    0.0027111
Osteoarthritis      0.0009610
[3 1 0 2]
Image path: ./drive/MyDrive/Colab Notebooks/data/valid/vG_Q9N.jpg
Top 4 results: 
class_id            probability
-------------------------------
Pneumonia           95.4764187
NoPneumonia         4.5171667
NoOsteoarthritis    0.0049002
Osteoarthritis      0.0015113
[3 1 0 2]
Image path: ./drive/MyDrive/Colab Notebooks/data/valid/5KFrPc.jpg
Top 4 results: 
class_id            probability
-------------------------------
Pneumonia           99.8205900
NoPneumonia         0.1469963
NoOsteoarthritis    0.0259077
Osteoarthritis      0.0065061
[3 1 0 2]
Image path: ./drive/MyDrive/Colab Notebooks/data/valid/0S_ZY_.jpg
Top 4 results: 
class_id            probability
-------------------------------
Pneumonia           99.9803245
NoPneumonia         0.0143364
NoOsteoarthritis    0.0031209
Osteoarthr